In [57]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)



import numpy as np
import pandas as pd
import seaborn as sb
sb.set_style("dark")
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

train_df_org = pd.read_csv('kaggle/input/digit-recognizer/train.csv',  nrows=300)
test_df_org = pd.read_csv('kaggle/input/digit-recognizer/test.csv',  nrows=300)

print("train_df_org shape ", train_df_org.shape) #  (1000, 785)
print("test_df_org shape ", test_df_org.shape) #  (300, 784)


train_df_org.head()

train_df_org shape  (300, 785)
test_df_org shape  (300, 784)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
# In test df there should not be any label column.
# lets see it.
test_df_org.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# Creating dataframe to contain all columns starting from "pizel0" till end
# note the first column is 'label'
mnist_train_df = train_df_org.loc[:, "pixel0":]

# print(mnist_train_df.shape) # (1000, 784)

# dataframe to contain only 'label' column
mnist_train_label = train_df_org.loc[:, "label"]
# print(mnist_train_label.shape) # (1000,)


In [60]:


import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import metrics
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Lambda, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D
from tensorflow.keras.optimizers import Adadelta
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler

In [61]:
train_df_org = pd.read_csv('kaggle/input/digit-recognizer/train.csv',  nrows=1000)

mnist_train_df = train_df_org.loc[:, "pixel0":]
# print(mnist_train_df.shape) # (1000, 784)

# Now dataframe with only 'label' column
mnist_train_label = train_df_org.loc[:, "label"]
# print(mnist_train_label.shape) # (1000,)

In [62]:
# Now convert the dataframes into numpy arrays
mnist_train_array = np.array(mnist_train_df)
mnist_train_label_array = np.array(mnist_train_label)

# Now need to do a little reshaping to get the structure which the model will expect to receive going forward
mnist_train_array = mnist_train_array.reshape(mnist_train_array.shape[0], 28, 28, 1)
print(mnist_train_array.shape, mnist_train_label_array.shape)
print(mnist_train_label_array)

(1000, 28, 28, 1) (1000,)
[1 0 1 4 0 0 7 3 5 3 8 9 1 3 3 1 2 0 7 5 8 6 2 0 2 3 6 9 9 7 8 9 4 9 2 1 3
 1 1 4 9 1 4 4 2 6 3 7 7 4 7 5 1 9 0 2 2 3 9 1 1 1 5 0 6 3 4 8 1 0 3 9 6 2
 6 4 7 1 4 1 5 4 8 9 2 9 9 8 9 6 3 6 4 6 2 9 1 2 0 5 9 2 7 7 2 8 8 5 0 6 0
 0 2 9 0 4 7 7 1 5 7 9 4 6 1 5 7 6 5 0 4 8 7 6 1 8 7 3 7 3 1 0 3 4 5 4 0 5
 4 0 3 5 1 0 8 3 7 0 9 6 6 9 5 4 6 9 3 5 4 2 4 8 7 7 5 8 8 8 2 6 9 3 1 0 4
 1 5 9 0 6 2 1 3 0 6 0 0 8 3 2 0 0 6 0 0 4 7 2 7 1 9 9 3 9 8 4 6 6 5 3 8 1
 8 7 1 3 7 6 3 6 3 6 3 2 3 2 2 7 9 2 3 2 7 5 5 8 8 2 0 1 4 0 6 3 7 1 1 1 4
 7 0 2 9 2 0 5 6 0 8 9 6 2 0 0 7 2 0 4 2 0 9 1 6 9 3 0 0 2 0 6 8 4 0 7 2 1
 9 5 2 4 8 5 2 9 7 9 2 9 7 4 9 3 2 7 3 6 3 6 8 8 3 7 0 9 2 7 9 0 5 4 5 8 4
 3 3 1 7 8 9 7 6 2 1 7 0 5 6 5 2 9 5 4 6 2 2 2 9 0 7 7 2 2 6 3 4 2 0 5 9 6
 2 1 9 0 6 0 4 8 4 3 1 5 4 2 9 5 7 3 1 5 4 5 3 7 3 8 6 2 4 6 1 1 4 0 0 5 8
 6 7 4 2 8 0 2 5 4 8 3 0 6 4 8 6 4 1 8 1 5 4 9 4 3 2 0 5 0 7 9 2 9 8 9 6 5
 2 4 4 6 4 8 4 1 7 5 8 9 5 9 3 2 5 8 2 2 7 2 8 4 1 9 3 6 0 2 2 9 1 2 7 2 1

In [63]:
num_of_digit_classes = mnist_train_label_array.max() - mnist_train_label_array.min() + 1
mnist_train_label_array = to_categorical(mnist_train_label_array, num_classes=num_of_digit_classes)
print('Shape of ytrain after encoding and converting to categorical values ', mnist_train_label_array.shape)
print(mnist_train_label_array)



Shape of ytrain after encoding and converting to categorical values  (1000, 10)
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [64]:
def run_model(input_shape=(28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))
    return model

def run_model_compilation(model, optimizer='adam', loss='categorical_crossentropy'):
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

def run_model_training(model, train, test, epochs, split):
    history=model.fit(train, test, shuffle=True, epochs=epochs, validation_split=split )
    return history

In [65]:
# Now actually train the model implementing all the functions above
cnn_model = run_model((28,28, 1))
run_model_compilation(cnn_model, 'adam', 'categorical_crossentropy')

# Taining the model specifying epocs
model_history = run_model_training(cnn_model, mnist_train_array, mnist_train_label_array, 4, 0.2)

Epoch 1/4
25/25 [==============================] - 2s 74ms/step - loss: 2.3130 - accuracy: 0.3212 - val_loss: 1.8006 - val_accuracy: 0.3500
Epoch 2/4
25/25 [==============================] - 2s 74ms/step - loss: 0.8024 - accuracy: 0.7462 - val_loss: 1.1928 - val_accuracy: 0.6000
Epoch 3/4
25/25 [==============================] - 2s 91ms/step - loss: 0.4358 - accuracy: 0.8725 - val_loss: 0.7450 - val_accuracy: 0.7450
Epoch 4/4
25/25 [==============================] - 2s 87ms/step - loss: 0.3328 - accuracy: 0.9025 - val_loss: 0.4628 - val_accuracy: 0.8700


In [66]:
# Now like the training data transformation to array I do the same with test data
# Convert the test dataframes into numpy arrays
mnist_test_array = np.array(test_df_org)
print("mnist_test_array.shape after converting to array ", mnist_test_array.shape)

# And also like the training data, need to do reshaping to get the structure which the model will expect to receive going forward
mnist_test_array = mnist_test_array.reshape(mnist_test_array.shape[0], 28, 28, 1)
print("mnist_test_array.shape after reshaping ", mnist_test_array.shape)

mnist_test_array.shape after converting to array  (300, 784)
mnist_test_array.shape after reshaping  (300, 28, 28, 1)


## Final Model Predictions and creating output file for Submission

In [67]:
final_predictions = cnn_model.predict(mnist_test_array)

prediction_test_array = []

for i in final_predictions:
    prediction_test_array.append(np.argmax(i))

In [68]:
submission = pd.DataFrame({
    'ImageId': test_df_org.index+1,
    'Label': prediction_test_array
})

submission.to_csv('final_submission.csv', index=False)
